In [ ]:
!unzip -q /content/drive/MyDrive/합성데이터_기반_.zip

In [ ]:
%pip install ultralytics
%pip install scikit-learn

In [ ]:
import os
import cv2
import yaml
import torch
import warnings
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from ultralytics import YOLO
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

In [ ]:
SEED = 42
BATCH_SIZE = 8
MODEL = "project_name"

In [ ]:
if os.path.exists("../data/yolo"):
    shutil.rmtree("../data/yolo")

if not os.path.exists("../data/yolo/train"):
    os.makedirs("../data/yolo/train")

if not os.path.exists("../data/yolo/valid"):
    os.makedirs("../data/yolo/valid")

if not os.path.exists("../data/yolo/test"):
    os.makedirs("../data/yolo/test")

if not os.path.exists("../results"):
    os.makedirs("../results")

In [ ]:
def make_yolo_dataset(image_paths, txt_paths, type="train"):
    for image_path, txt_path in tqdm(zip(image_paths, txt_paths if not type == "test" else image_paths), total=len(image_paths)):
        source_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image_height, image_width, _ = source_image.shape

        target_image_path = f"../data/yolo/{type}/{os.path.basename(image_path)}"
        cv2.imwrite(target_image_path, source_image)

        if type == "test":
            continue

        with open(txt_path, "r") as reader:
            yolo_labels = []
            for line in reader.readlines():
                line = list(map(float, line.strip().split(" ")))
                class_name = int(line[0])
                x_min, y_min = float(min(line[5], line[7])), float(min(line[6], line[8]))
                x_max, y_max = float(max(line[1], line[3])), float(max(line[2], line[4]))
                x, y = float(((x_min + x_max) / 2) / image_width), float(((y_min + y_max) / 2) / image_height)
                w, h = abs(x_max - x_min) / image_width, abs(y_max - y_min) / image_height
                yolo_labels.append(f"{class_name} {x} {y} {w} {h}")

        target_label_txt = f"../data/yolo/{type}/{os.path.basename(txt_path)}"
        with open(target_label_txt, "w") as writer:
            for yolo_label in yolo_labels:
                writer.write(f"{yolo_label}\n")

In [ ]:
image_paths = sorted(glob("/content/train/*.png"))
txt_paths = sorted(glob("/content/train/*.txt"))

train_images_paths, valid_images_paths, train_txt_paths, valid_txt_paths = train_test_split(image_paths, txt_paths, test_size=0.1, random_state=SEED)

make_yolo_dataset(train_images_paths, train_txt_paths, "train")
make_yolo_dataset(valid_images_paths, valid_txt_paths, "valid")
make_yolo_dataset(sorted(glob("/content/test/*.png")), None, "test")

In [ ]:
with open("./classes.txt", "r") as reader:
    lines = reader.readlines()
    classes = [line.strip().split(",")[1] for line in lines]

yaml_data = {
              "names": classes,
              "nc": len(classes),
              "path": "/data/yolo/",
              "train": "train",
              "val": "valid",
              "test": "test"
            }

with open("./yolocustom.yaml", "w") as writer:  # 경로 오류 발생시 절대 경로로 수정하세요.
    yaml.dump(yaml_data, writer)

In [ ]:
model = YOLO("yolov8x")
results = model.train(
    data="./yolocustom.yaml",
    imgsz=1024,
    epochs=200,
    batch=BATCH_SIZE,
    patience=5,
    workers=16,
    device=0,
    exist_ok=True,
    project=f"{MODEL}",
    name="train",
    seed=SEED,
    pretrained=False,
    resume=False,
    optimizer="AdamW",
    lr0=1e-3,
    val=True,
    cache=True,
    mixup = 0.5   # use mix-up Augmentation
    )

Ultralytics YOLOv8.0.125 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=./yolocustom.yaml, epochs=200, patience=5, batch=8, imgsz=1024, save=True, save_period=-1, cache=True, device=0, workers=16, project=project_name, name=train, exist_ok=True, pretrained=False, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=Fa

In [ ]:
def get_test_image_paths(test_image_paths):
    for i in range(0, len(test_image_paths), BATCH_SIZE):
        yield test_image_paths[i:i+BATCH_SIZE]

In [ ]:
model = YOLO(f"{MODEL}/train/weights/best.pt")
test_image_paths = glob("/data/yolo/test/*.png")

for i, image in tqdm(enumerate(get_test_image_paths(test_image_paths)), total=int(len(test_image_paths)/BATCH_SIZE)):
    model.predict(image, imgsz=(1024, 1024), iou=0.2, conf=0.5, save_conf=True, save=False, save_txt=True, project=f"{MODEL}", name="predict",
                  exist_ok=True, device=0, augment=True, verbose=False)
    if i % 5 == 0:
        clear_output(wait=True)

 99%|█████████▉| 421/425 [07:16<00:03,  1.25it/s]Results saved to project_name/predict
3376 labels saved to project_name/predict/labels
 99%|█████████▉| 422/425 [07:17<00:02,  1.25it/s]Results saved to project_name/predict
3384 labels saved to project_name/predict/labels
100%|█████████▉| 423/425 [07:18<00:01,  1.25it/s]Results saved to project_name/predict
3392 labels saved to project_name/predict/labels
100%|█████████▉| 424/425 [07:19<00:00,  1.23it/s]Results saved to project_name/predict
3400 labels saved to project_name/predict/labels
100%|██████████| 425/425 [07:19<00:00,  1.04s/it]


In [ ]:
import pandas as pd

from tqdm import tqdm
from glob import glob

In [ ]:
submit = pd.read_csv('./sample_submission.csv', header=0)
submit.columns

Index(['file_name', 'class_id', 'confidence', 'point1_x', 'point1_y',
       'point2_x', 'point2_y', 'point3_x', 'point3_y', 'point4_x', 'point4_y'],
      dtype='object')

In [ ]:
width = 1920
height = 1080

# bbox -> 상대좌표계, width, height -> 이미지 해상도
def convert_yolo_to_cv2(width, height, bbox):
    x, y, w, h = bbox
    xmin = int((x - w / 2) * width)
    xmax = int((x + w / 2) * width)
    ymin = int((y - h / 2) * height)
    ymax = int((y + h / 2) * height)
    return xmin, ymin, xmax, ymax

for txt_file in tqdm(glob("/content/project_name/predict/labels/*.txt")):
    # window환경에서 진행하는 경우 split('\\')로 변경 필요
    file_name= txt_file.split('/')[-1].replace('.txt','.png')
    with open(txt_file) as f:
        for ann in f:
            # text결과값의 경우 cls, x, y, w, h, conf 순으로 작업
            cls, x,y,w,h,conf = [float(temp) for temp in ann.split()]
            xmin, ymin, xmax, ymax = convert_yolo_to_cv2(width, height,[x,y,w,h])
            if conf <0.5:
                continue
            # file_name, class, confidence, left, top, right, top, right, bottom, left, bottm
            submit.loc[len(submit)] = [file_name, int(cls), conf, xmin, ymin, xmax, ymin, xmax, ymax, xmin, ymax]

100%|██████████| 3400/3400 [00:15<00:00, 222.08it/s]


In [ ]:
submit.to_csv('submit.csv', index=False, encoding='utf-8')